<a href="https://colab.research.google.com/github/DmitryKutsev/cross-document_coreference/blob/main/classifiers/rule_ml_pair_pron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [349]:
pip install pymorphy2[fast]

     |████████████████████████████████| 61kB 3.6MB/s 
     |████████████████████████████████| 8.2MB 7.1MB/s 
     |████████████████████████████████| 378kB 37.2MB/s 
  Created wheel for DAWG: filename=DAWG-0.8.0-cp37-cp37m-linux_x86_64.whl size=865381 sha256=fd32f8d23245dd7ec1f0642cb8f8570222382d288cd1d448e314c9724b53dcda
  Stored in directory: /root/.cache/pip/wheels/3d/1f/f0/a5b1f9d02e193c997d252c33d215f24dfd7a448bc0166b2a12
Successfully built DAWG


## **Импорт**

In [336]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


In [337]:
from IPython.display import SVG, display, HTML

In [338]:
from google.colab import drive
drive.mount('./content', force_remount=False)

Drive already mounted at ./content; to attempt to forcibly remount, call drive.mount("./content", force_remount=True).


In [339]:
import spacy
import numpy as np

import gensim
from gensim.models import Word2Vec 
from gensim.models import FastText

from nltk.tokenize import RegexpTokenizer
from nltk import  sent_tokenize
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

In [340]:
from graphviz import Source
from sklearn import tree, svm, linear_model
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier


In [341]:
!ls
import os
for i in os.listdir('content/'):
  print(i)

annotations.csv       hv-info_gepatit2.txt	 sample_data
annotation_table.csv  neuroleptic_antidepr2.txt
content		      rakpobedim_ru2.txt
.file-revisions-by-id
.shortcut-targets-by-id
MyDrive
.Trash


In [342]:
import csv
import re
from google.colab import files
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

## **Обработка данных**

In [343]:
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/neuroleptic_antidepr2.txt
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/rakpobedim_ru2.txt
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/hv-info_gepatit2.txt

--2021-05-25 09:47:11--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/neuroleptic_antidepr2.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11655409 (11M) [text/plain]
Saving to: ‘neuroleptic_antidepr2.txt.1’

neuroleptic_antidep 100%[===================>]  11.12M  53.1MB/s    in 0.2s    

2021-05-25 09:47:12 (53.1 MB/s) - ‘neuroleptic_antidepr2.txt.1’ saved [11655409/11655409]

--2021-05-25 09:47:12--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/rakpobedim_ru2.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:

In [344]:
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/coref_df_anns60k.csv
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/annotation_table.csv

--2021-05-25 09:47:13--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/coref_df_anns60k.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2021-05-25 09:47:13 ERROR 404: Not Found.

--2021-05-25 09:47:14--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/annotation_table.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2100361 (2.0M) [text/plain]
Saving to: ‘annotation_table.csv.1’

annotation_table.cs 100%[===================>]   2.00M  --.-KB/s    in 0.

In [345]:
handler = open('neuroleptic_antidepr2.txt', 'r', encoding='utf-8')
my_txt = handler.read()
handler.close()

handler = open('hv-info_gepatit2.txt', 'r', encoding='utf-8')
my_txt += handler.read()
handler.close()

handler = open('rakpobedim_ru2.txt', 'r', encoding='utf-8')
my_txt += handler.read()
handler.close()

In [346]:
posts = my_txt.split('----')

In [350]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [352]:
morph.parse('он')[0].normal_form

'он'

## **Обучение векторизатора**

In [353]:
tokenizer = RegexpTokenizer(r'\w+')
sentences_tokenized = [w.lower() for w in posts]


sentences_tokenized = [tokenizer.tokenize(morph.parse(i)[0].normal_form) for i in sentences_tokenized]
ft = FastText(sentences_tokenized, window=5, min_count=5, workers=4,sg=1)

Обработка битых символов, чтобы pandas мог открыть таблицу.

In [354]:
count = 0
with open('annotation_table.csv') as csvfile:
  with open('annotations.csv', 'w') as write_file:
    writer = csv.writer(write_file, delimiter='\t')
    for new_row in csvfile:
      count +=1
      # print(i.split('\t'))
      if len(new_row.split('\t')) > 19:
        new_row = new_row.split('\t')[:20]
        new_row[-1] = new_row[-1][:-1] # cut \n
      writer.writerow(new_row)
count

27019

In [355]:
# # with open('coref_df_anns60k.csv') as csvfile:
# with open('new_anns60k.csv') as csvfile:
#   reader = csv.reader(csvfile, delimiter='\t')
#   with open('new_df_anns60k.csv', 'w') as write_file:
#     writer = csv.writer(write_file, delimiter='\t')
#     c = 0
#     for row in reader:
#       c +=1
#       new_row = [i if i and not re.match(r'\t+?', i) else 0 for i in row]
#       if len(new_row) > 9:
#         new_row = new_row[:9]
#       else:
#         new_row = [0 for i in range(9)]
#       writer.writerow(new_row)
# c


In [356]:
# !rm new_anns60k.csv.1 new_anns60k.csv.2 new_df_anns60k.csv new_anns60k.csv
!ls
# !rm annotation_table.csv.2 annotation_table.csv.1 annotation_table.csv annotations.csv

annotations.csv		hv-info_gepatit2.txt	     rakpobedim_ru2.txt
annotation_table.csv	hv-info_gepatit2.txt.1	     rakpobedim_ru2.txt.1
annotation_table.csv.1	neuroleptic_antidepr2.txt    sample_data
content			neuroleptic_antidepr2.txt.1


In [357]:
# anns_df = pd.read_csv('coref_df_anns60k.csv', delimiter='\t', error_bad_lines=False, header=None)
anns_df = pd.read_csv('annotations.csv', delimiter='\t')
anns_df

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER
0,обсуждаем,0,1,0.0,0.0,0.0,NaN,NaN,NaN,1per,VERB,ROOT,обсуждаем,NaN,NaN,NaN,VERB,0,0.0,plur
1,препарат,0,1,0.0,0.0,1.0,inan,nomn,masc,NaN,NOUN,obj,обсуждаем,NaN,NaN,NaN,VERB,0,0.0,sing
2,рисперидон,0,1,0.0,0.0,2.0,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,1,2.0,sing
3,рисполепт,0,1,0.0,0.0,3.0,inan,gent,femn,NaN,NOUN,parataxis,рисперидон,anim,nomn,masc,NOUN,1,3.0,plur
4,сперидан,0,1,0.0,0.0,4.0,NaN,NaN,masc,NaN,PRTS,conj,рисполепт,inan,gent,femn,NOUN,1,4.0,sing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27013,и,0,22,68.0,0.0,2385.0,NaN,NaN,NaN,NaN,CONJ,cc,пошлины,inan,nomn,femn,NOUN,0,0.0,NaN
27014,таможенные,0,22,68.0,0.0,2386.0,NaN,nomn,NaN,NaN,ADJF,amod,пошлины,inan,nomn,femn,NOUN,0,0.0,plur
27015,пошлины,0,22,68.0,0.0,2387.0,inan,nomn,femn,NaN,NOUN,nsubj,есть,NaN,NaN,NaN,INFN,0,0.0,plur
27016,как,0,22,68.0,0.0,2388.0,NaN,NaN,NaN,NaN,CONJ,advmod,есть,NaN,NaN,NaN,INFN,0,0.0,NaN


In [358]:
tok_list = []
for token in anns_df.TOKEN:
  try:
    tok_list.append(ft.wv.__getitem__(token.lower()))
  except (KeyError, AttributeError):
    tok_list.append(0)
    # print(token, 'here')
anns_df['TOKEN_VECT'] = tok_list


tok_list = []
for token in anns_df.HEAD:
  try:
    tok_list.append(ft.wv.__getitem__(token.lower()))
  except (KeyError, AttributeError):
    tok_list.append(0)
    # print(token, 'here')
anns_df['HEAD_VECT'] = tok_list

In [359]:
# tok_list = []
# for sent in 
# for token in anns_df.HEAD:
#   try:
#     tok_list.append(ft.wv.__getitem__(token.lower()))
#   except KeyError:
#     tok_list.append(0)
#     print(token, 'here')
# anns_df['TOKEN_VECT'] = tok_list

In [360]:
anns_df = anns_df.loc[(anns_df['TOPIC_NUM'].isin(list(range(30)))) | (anns_df['SENT_NUM'].isin(list(range(10000))))]
anns_df = anns_df.reset_index().drop(['index'], axis=1)

In [361]:
topics = list(set(anns_df.TOPIC_NUM.values))
topics
sent_vect_list = []
# номера предложений топика
for topic_num in topics:

  sents_in_topic = list(set(anns_df.loc[anns_df['TOPIC_NUM'] == topic_num].SENT_NUM))
  topic_len = len(anns_df.loc[anns_df['TOPIC_NUM'] == topic_num])
  for sent_num in sents_in_topic:
    tok_vects = anns_df.loc[(anns_df['TOPIC_NUM'] == topic_num) & (anns_df['SENT_NUM'] == sent_num)].TOKEN_VECT
    try:
      sent_vect = sum(list(tok_vects))/len(tok_vects)
    except ZeroDivisionError:
      sent_vect = 0
      print(anns_df.loc[(anns_df['SENT_NUM'] == sent_num)])
    for i in range(len(tok_vects)):
      sent_vect_list.append(sent_vect)
len(sent_vect_list)
# если не класть нулевых предложений - будет длина датафрейма, как он раньше импортировлся, но поздно.

26497

In [362]:
anns_df['SENT_VECT'] = sent_vect_list

Проверим данные

In [363]:
anns_df.SENT_NUM[18]

1.0

In [364]:
# topics = list(set(df.TOPIC_NUM.values)) 
# for topic_num in topics:
#   # if topic_num == 2:
#   #   break
#   # извлекаем предложения топика
#   sents_in_topic = list(set(df.loc[df['TOPIC_NUM'] == topic_num ].SENT_NUM))
#   for sent_num in sents_in_topic:
#     sent = df.loc[(df['TOPIC_NUM'] == topic_num)  & (df['SENT_NUM'] == sent_num)]

In [365]:
# vvv = 0
# with open('content/MyDrive/coref_fasttext60k_1.csv') as csvfile:
#   with open('new_coref_df_anns60k.csv', 'w') as write_file:
#     writer = csv.writer(write_file, delimiter='\t')
#     for new_row in csvfile:
#       vvv +=1
#       # print(i.split('\t'))
#       if len(new_row.split('\t')) > 9:
#         new_row = new_row.split('\t')[:9]
#       writer.writerow(new_row)
# vvv

In [366]:
# cor_df = pd.read_csv('content/MyDrive/coref_fasttext60k_1.csv', delimiter=' ')
# cor_df

In [367]:
# cosine_similarity([np.matrix(cor_df.TOKEN_VECT[2]).A[0]], [np.matrix(cor_df.TOKEN_VECT[3]).A[0]])[-1][-1]

# **VECTORS HANDLING**

Изменяем значения векторов предложений, главного слова группы, зависимых слов группы, и токена на косинусную дистанцию с семантической осью.

Сначала, изменим строковое представление на массив.

In [368]:
# cor_df.TOKEN_VECT = [np.matrix(i).A[0] for i in cor_df.TOKEN_VECT]
# cor_df.HEAD_VECT = [np.matrix(i).A[0] for i in cor_df.HEAD_VECT]
# cor_df.SENT_VECT = [np.matrix(i).A[0] for i in cor_df.SENT_VECT]
# cor_df.HEAD_CHILDS_VECT = [np.matrix(i).A[0] for i in cor_df.HEAD_CHILDS_VECT]


Добавим размеченные данные, и посмотрим, что получилось.

In [369]:
# cor_df['NER'] = anns_df['NER']
# cor_df['COREFERENCE_CLUSTER'] = anns_df['COREFERENCE_CLUSTER']
# cor_df['ANN_TOKEN'] = anns_df['TOKEN']
# cor_df

In [370]:
# cor_df.loc()

Ограничим датафрейм теми данными, которые были размечены, чтобы была возможность провести оценку системы.

In [371]:
# df = anns_df.iloc[:15000,:]
df = anns_df.iloc[:17200,:]

Всего наименований препаратов размечено (это не обязательно антецеденты, любое название препарата).

In [372]:
len(df.loc[df['NER'] == 1])

581

Из них местоимений (прономинальные анафоры).

In [373]:
len(df.loc[df['NER'] == 3])

113

И номинативные анафоры существительные, такие как "препарат" и "лекарство". В основном это они и есть.

In [374]:
len(df.loc[df['NER'] == 2])

215

In [439]:
len(df.loc[df['NER'].isin([2,3])])

328

In [375]:
len(df.loc[df['NER'].isin([1,2,3])])

909

In [376]:
# df['NER'] = [int(i) for i in df['NER']]
# df['COREFERENCE_CLUSTER'] = [int(i) for i in df['COREFERENCE_CLUSTER']]

Посмотрим на данные.

In [377]:
df.loc[df['NER'].isin([1,2,3])]

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
2,рисперидон,0,1,0.0,0.0,2.0,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,1,2.0,sing,"[0.90845776, 0.7599383, 0.7973972, 0.06410726,...","[-0.8513295, 1.1935784, 1.0183439, 1.1038748, ...","[0.0959547, 0.32521266, 0.077532694, -0.026430..."
3,рисполепт,0,1,0.0,0.0,3.0,inan,gent,femn,NaN,NOUN,parataxis,рисперидон,anim,nomn,masc,NOUN,1,3.0,plur,"[0.41658387, 0.64925975, 0.794993, -0.20932783...","[0.90845776, 0.7599383, 0.7973972, 0.06410726,...","[0.0959547, 0.32521266, 0.077532694, -0.026430..."
4,сперидан,0,1,0.0,0.0,4.0,NaN,NaN,masc,NaN,PRTS,conj,рисполепт,inan,gent,femn,NOUN,1,4.0,sing,"[0.74793136, 0.70079494, 0.64134437, -0.061814...","[0.41658387, 0.64925975, 0.794993, -0.20932783...","[0.0959547, 0.32521266, 0.077532694, -0.026430..."
21,лекарством,0,1,1.0,1.0,21.0,inan,ablt,neut,NaN,NOUN,conj,нет,NaN,NaN,NaN,PRED,2,0.0,sing,"[-0.4091129, 0.4388335, 0.4519824, 0.2503013, ...","[-0.05246899, -0.39525706, 0.0034962893, 0.261...","[0.0959547, 0.32521266, 0.077532694, -0.026430..."
142,рисполепта,0,1,1.0,13.0,142.0,inan,nomn,femn,NaN,NOUN,nmod,дозировку,inan,accs,femn,NOUN,1,142.0,sing,"[0.42524558, 0.37402976, 0.93385345, -0.154084...","[0.046840813, 0.5573126, 0.3759949, 0.48286912...","[0.26892328, 0.34796563, 0.016341658, 0.110777..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14807,абраксан,1,15,18.0,3.0,2215.0,anim,gent,femn,NaN,NOUN,obj,спросите,NaN,NaN,NaN,VERB,1,2215.0,plur,"[0.49983642, 1.073201, 0.2674687, 0.045585226,...","[-0.4549187, -0.07701046, -0.4658066, -0.45628...","[0.29813343, 0.38386613, 0.10309626, 0.1697261..."
14809,гемцитабином,1,15,18.0,3.0,2217.0,inan,ablt,masc,NaN,NOUN,nmod,абраксан,anim,gent,femn,NOUN,1,2217.0,sing,"[-0.04373059, 1.1794723, -0.035744626, 0.19495...","[0.49983642, 1.073201, 0.2674687, 0.045585226,...","[0.29813343, 0.38386613, 0.10309626, 0.1697261..."
14856,фолфиринокс,1,15,18.0,8.0,2264.0,inan,nomn,masc,NaN,NOUN,nmod,схема,inan,nomn,femn,NOUN,1,2264.0,sing,"[0.27262083, 0.3407862, 0.20147441, -0.0261929...","[0.70859116, 0.49130666, 0.9105393, -0.9568749...","[0.26331103, 0.27965796, -0.06875322, -0.01667..."
14887,иринотекану,1,15,19.0,0.0,2295.0,inan,datv,masc,NaN,NOUN,nmod,аннотации,inan,nomn,femn,NOUN,1,2295.0,sing,"[0.22175883, 0.69562685, 0.1413071, -0.0980190...","[0.18104157, 0.10558991, 0.32197434, -0.059847...","[0.26331103, 0.27965796, -0.06875322, -0.01667..."


# **SEMANTIC AXIS**

Реализация метода семантических осей.
Сначала, выбираем первые предложения каждого топика. Посмотрим на них.

In [378]:
axis_df = df.query('POST_NUM == 0 and TOKEN_NUM == 2 and SENT_NUM == 0')
axis_df

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
2,рисперидон,0,1,0.0,0.0,2.0,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,1,2.0,sing,"[0.90845776, 0.7599383, 0.7973972, 0.06410726,...","[-0.8513295, 1.1935784, 1.0183439, 1.1038748, ...","[0.0959547, 0.32521266, 0.077532694, -0.026430..."
576,зелдокс,0,2,0.0,0.0,2.0,inan,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,1,2.0,sing,"[0.33465928, 0.7358735, 0.39308316, 0.21810421...","[-0.8513295, 1.1935784, 1.0183439, 1.1038748, ...","[0.28294712, 0.23582223, 0.1933776, -0.0144803..."
1870,оланзапин,0,3,0.0,0.0,2.0,inan,gent,femn,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,1,2.0,plur,"[0.69123894, 0.7543587, 0.3582821, -0.33660454...","[-0.8513295, 1.1935784, 1.0183439, 1.1038748, ...","[0.18222502, 0.25052592, 0.1167401, 0.04071825..."
5555,галоперидол,0,4,0.0,0.0,2.0,inan,nomn,masc,NaN,NOUN,obj,обсуждаем,NaN,NaN,NaN,VERB,1,2.0,sing,"[0.5619719, 1.1833849, 0.5102037, -0.11594948,...","[-0.10694176, 0.5883629, -0.29518884, -0.47235...","[0.21754177, 0.12213363, 0.07084999, 0.0018490..."
6200,тиаприд,0,5,0.0,0.0,2.0,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,1,3.0,sing,"[0.5293834, 0.60194075, 0.9200476, -0.11615737...","[-0.8513295, 1.1935784, 1.0183439, 1.1038748, ...","[0.1654229, 0.12817962, 0.07227736, 0.02525708..."
8450,сероквель,0,6,0.0,0.0,2.0,inan,gent,femn,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,1,2.0,plur,"[0.4656885, 1.0141326, 0.6882786, -0.01875147,...","[-0.8513295, 1.1935784, 1.0183439, 1.1038748, ...","[0.2673824, 0.31761414, 0.21638454, 0.11968637..."
8541,день,0,7,0.0,0.0,2.0,inan,accs,masc,NaN,NOUN,ROOT,день,inan,accs,masc,NOUN,0,0.0,sing,"[-0.0064989817, 0.76366216, -0.6513326, 0.3287...","[-0.0064989817, 0.76366216, -0.6513326, 0.3287...","[0.12212184, 0.192821, 0.19801214, 0.13695534,..."
11117,500,0,8,0.0,0.0,2.0,NaN,NaN,NaN,NaN,NaN,nummod,мг,inan,nomn,masc,NOUN,0,0.0,NaN,"[-0.26605016, 0.30501804, 0.9514461, 0.5048218...","[0.31216827, 0.5124039, 0.82379, -0.24131414, ...","[0.19772822, 0.33661807, 0.25174907, 0.1309174..."
11787,капецитабин,0,10,0.0,0.0,2.0,anim,nomn,masc,NaN,NOUN,ROOT,капецитабин,anim,nomn,masc,NOUN,1,2.0,sing,"[0.15125047, 1.206354, -0.12286777, -0.1759417...","[0.15125047, 1.206354, -0.12286777, -0.1759417...","[0.2410063, 0.2747689, 0.17557833, 0.021017414..."
11806,иресса,0,11,0.0,0.0,2.0,NaN,NaN,NaN,NaN,GRND,nmod,отдам,NaN,NaN,NaN,VERB,1,2.0,NaN,"[-0.3009659, 0.63091695, 0.54779667, -0.367333...","[-0.30435267, 0.5013345, -0.8559268, -0.088319...","[0.2410063, 0.2747689, 0.17557833, 0.021017414..."


In [379]:
# sum(df.query('POST_NUM == 0 and TOKEN_NUM == 2 and SENT_NUM == 0').TOKEN_VECT)/len(df.query('POST_NUM == 0 and TOKEN_NUM == 2 and SENT_NUM == 0').TOKEN_VECT)

Суммируем верктора токенов, главных слов, предложения целиком, и зависимых слов. Нормализуем.

In [380]:

token_axe = sum(axis_df.TOKEN_VECT)/len(axis_df.TOKEN_VECT)
head_axe = sum(axis_df.HEAD_VECT)/len(axis_df.HEAD_VECT)
# head_childs_axe = sum(axis_df.HEAD_CHILDS_VECT)/len(axis_df.HEAD_CHILDS_VECT)
sent_axe = sum(axis_df.SENT_VECT)/len(axis_df.SENT_VECT)

In [381]:
df.TOKEN_VECT[710]

0

Функция, которая заменяет столбец с векторами на столбец со значениями косинусной дистанции с соответствующей семантической осью.


In [382]:
def make_similarity(df, col, axe):
  new_col = []
  for ind, vect in enumerate(df[col]):
    try:
      new_col.append(cosine_similarity([axe], [vect])[-1][-1])
    except ValueError:
      new_col.append(0)
  df[col] = new_col



In [383]:
make_similarity(df, 'TOKEN_VECT', token_axe)
make_similarity(df, 'HEAD_VECT', head_axe)
# make_similarity(df, 'HEAD_CHILDS_VECT', head_childs_axe)
make_similarity(df, 'SENT_VECT', sent_axe)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

In [384]:
df.TOKEN[1]

'препарат'

In [385]:
# df['SUM_VECT']= [(df.TOKEN_VECT[i] + df.SENT_VECT[i])/3 for i in range(len(df.SENT_VECT))]

Посмотрим на обработанные данные.

In [386]:
df

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
0,обсуждаем,0,1,0.0,0.0,0.0,NaN,NaN,NaN,1per,VERB,ROOT,обсуждаем,NaN,NaN,NaN,VERB,0,0.0,plur,0.495954,0.528017,0.952796
1,препарат,0,1,0.0,0.0,1.0,inan,nomn,masc,NaN,NOUN,obj,обсуждаем,NaN,NaN,NaN,VERB,0,0.0,sing,0.560645,0.528017,0.952796
2,рисперидон,0,1,0.0,0.0,2.0,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,1,2.0,sing,0.769174,0.865285,0.952796
3,рисполепт,0,1,0.0,0.0,3.0,inan,gent,femn,NaN,NOUN,parataxis,рисперидон,anim,nomn,masc,NOUN,1,3.0,plur,0.775658,0.556566,0.952796
4,сперидан,0,1,0.0,0.0,4.0,NaN,NaN,masc,NaN,PRTS,conj,рисполепт,inan,gent,femn,NOUN,1,4.0,sing,0.750261,0.634239,0.952796
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17195,теперь,1,20,4.0,1.0,606.0,NaN,NaN,NaN,NaN,ADVB,advmod,назначили,NaN,NaN,NaN,VERB,0,0.0,NaN,0.490155,0.575457,0.993070
17196,нам,1,20,4.0,1.0,607.0,NaN,datv,NaN,1per,NPRO,iobj,назначили,NaN,NaN,NaN,VERB,0,0.0,plur,0.536465,0.575457,0.993070
17197,назначили,1,20,4.0,1.0,608.0,NaN,NaN,NaN,NaN,VERB,conj,пошел,NaN,NaN,masc,VERB,0,0.0,plur,0.671096,0.567691,0.993070
17198,пить,1,20,4.0,1.0,609.0,NaN,NaN,NaN,NaN,INFN,xcomp,назначили,NaN,NaN,NaN,VERB,0,0.0,NaN,0.612565,0.575457,0.993070


# **ПРАВИЛОВЫЙ АЛГОРИТМ**

In [387]:
# df.SENT_NUM = [int(i) for i in df['SENT_NUM']]
# df.POST_NUM = [int(i) for i in df['POST_NUM']]
# # df.TOKEN_NUM = [float(i) for i in df['TOKEN_NUM']]
df.TOPIC_NUM = [int(i) for i in df['TOPIC_NUM']]
df.IS_ANSWER = [int(i) for i in df['TOPIC_NUM']]
# df.NER = [int(i) for i in df['NER']]

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [388]:
topics = list(set(df.TOPIC_NUM.values))
topics

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

In [389]:
df.loc[df['TOKEN_NUM'] == 344 ]

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
344,рисполепта,1,1,2.0,4.0,344.0,inan,nomn,femn,NaN,NOUN,nmod,приеме,inan,loct,masc,NOUN,1,344.0,sing,0.734191,0.494157,0.947810
918,вышло,2,2,6.0,3.0,344.0,NaN,NaN,neut,NaN,VERB,ROOT,вышло,NaN,NaN,neut,VERB,0,0.0,sing,0.524962,0.503916,0.874704
2212,в,3,3,5.0,8.0,344.0,NaN,NaN,NaN,NaN,PREP,case,использовании,inan,loct,neut,NOUN,0,0.0,NaN,0.471864,0.531794,0.906940
5897,и,4,4,6.0,7.0,344.0,NaN,NaN,NaN,NaN,CONJ,cc,выписывались,NaN,NaN,NaN,VERB,0,0.0,NaN,0.440320,0.500749,0.988290
6542,в,5,5,17.0,0.0,344.0,NaN,NaN,NaN,NaN,PREP,case,мозгу,inan,loc2,masc,NOUN,0,0.0,NaN,0.471864,0.429818,0.984281
8883,их,7,7,0.0,32.0,344.0,NaN,accs,NaN,3per,NPRO,obj,принимать,NaN,NaN,NaN,INFN,0,0.0,plur,0.425714,0.489115,0.698363
11513,отвратно,9,9,0.0,39.0,344.0,NaN,NaN,NaN,NaN,ADVB,advmod,перенесла,NaN,NaN,femn,VERB,0,0.0,NaN,0.551399,0.539211,0.977045
12936,будет,15,15,0.0,31.0,344.0,NaN,NaN,NaN,3per,VERB,advcl,проходить,NaN,NaN,NaN,INFN,0,0.0,sing,0.499137,0.598919,0.972264
15278,ковид,16,16,9.0,0.0,344.0,inan,nomn,masc,NaN,NOUN,conj,наименование,inan,nomn,neut,NOUN,0,0.0,sing,0.581907,0.504518,0.967173
16465,сказал,18,18,7.0,2.0,344.0,NaN,NaN,masc,NaN,VERB,acl:relcl,парень,anim,nomn,masc,NOUN,0,0.0,sing,0.475929,0.481758,0.991081


In [390]:

topic_num = 1
topic_df = df.loc[df['TOPIC_NUM'] == topic_num]
# номера предложений топика
sents_in_topic = list(set(df.loc[df['TOPIC_NUM'] == topic_num ].SENT_NUM))
sents_in_topic
# достать пост
df.loc[(df['TOPIC_NUM'] == 1) & (df['POST_NUM'] == 1)]
# достать конкретное предложение конкретного поста
df.loc[(df['TOPIC_NUM'] == 1) & (df['POST_NUM'] == 1) & (df['SENT_NUM'] == 1)]
# достать токен из предложение конкретного поста
df.loc[(df['TOPIC_NUM'] == 1) & (df['POST_NUM'] == 1) & (df['SENT_NUM'] == 0)]


,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
5,не,1,1,1.0,0.0,5.0,NaN,NaN,NaN,NaN,PRCL,advmod,знаю,NaN,NaN,NaN,VERB,0,0.0,NaN,0.437783,0.474300,0.952796
6,знаю,1,1,1.0,0.0,6.0,NaN,NaN,NaN,1per,VERB,ROOT,знаю,NaN,NaN,NaN,VERB,0,0.0,sing,0.532945,0.474300,0.952796
7,стоило,1,1,1.0,0.0,7.0,NaN,NaN,neut,NaN,VERB,ccomp,знаю,NaN,NaN,NaN,VERB,0,0.0,sing,0.492581,0.474300,0.952796
8,ли,1,1,1.0,0.0,8.0,NaN,NaN,NaN,NaN,CONJ,advmod,стоило,NaN,NaN,neut,VERB,0,0.0,NaN,0.514900,0.474943,0.952796
9,мне,1,1,1.0,0.0,9.0,NaN,datv,NaN,1per,NPRO,iobj,стоило,NaN,NaN,neut,VERB,0,0.0,sing,0.499491,0.474943,0.952796
10,писать,1,1,1.0,0.0,10.0,NaN,NaN,NaN,NaN,INFN,csubj,стоило,NaN,NaN,neut,VERB,0,0.0,NaN,0.495525,0.474943,0.952796


In [391]:
for i in df.loc[(df['TOPIC_NUM'] == 1) & (df['POST_NUM'] == 1) & (df['SENT_NUM'] == 0)].values:
  print(i[-5])

0.0
0.0
0.0
0.0
0.0
0.0


In [392]:
# анафоры
df.loc[(df['TOPIC_NUM'] == 1) & (df['POST_NUM'] == 1) & (df['SENT_NUM'] == 1)
& (df['NER'] > 1)].values[0]

array(['лекарством', 1, 1, 1.0, 1.0, 21.0, 'inan', 'ablt', 'neut', nan,
       'NOUN', 'conj', 'нет', nan, nan, nan, 'PRED', 2, 0.0, 'sing',
       0.4843989610671997, 0.5488108992576599, 0.9527955055236816],
      dtype=object)

In [393]:
# sents_in_topic 
# topic_num
# df = df.reset_index()

In [394]:
# knn_clf = KNeighborsClassifier(n_neighbors=10, weights='distance',algorithm='ball_tree',leaf_size=5, metric='minkowski', p=1).fit(X, y)
# knn_clf.score(X_test, y_test)

In [395]:
def rule_eval(my_X, my_y):
  
  tp = []
  fp = []
  tn = []
  fn = []

  for i in range(len(my_X)):
    if my_X[i] != 0:
      if my_X[i] == my_y[i]:
        tp.append(i)
      else:
        fp.append(i)
    # берем только те синглтоны, которые относятся к номинативным или прономинальным группам.
    elif my_X[i] == 0:
      if my_X[i] == my_y[i]:
        tn.append(i)
      else:
        fn.append(i)
  try:
    P = len(tp)/(len(tp) + len(fp))
  except ZeroDivisionError:
    P = 0
  try:
    R = len(tp)/(len(tp) + len(fn))
  except ZeroDivisionError:
    R = 0
  try:
    F = 2*P*R/(P+R)
  except ZeroDivisionError:
    F = 0

  print(len(tp), ' true positive')
  print(len(fp), ' false positive')
  print(len(tn), ' true negative')
  print(len(fn), ' false negative')
  print('---------')
  print(P,' presicion')
  print(R, ' recall')
  print(F, ' f-score')


In [586]:
df.loc[(df['POS'] == 5) &  (df['PERSON'] == 1)].iloc[30:40, :]

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
2875,он,3,3,15.0,2.0,1007.0,-1,0,0,1,5,12,489,-1,-1,-1,0,3,987.0,1,0.493189,0.505489,0.979014
2888,его,3,3,15.0,3.0,1020.0,-1,3,0,1,5,21,495,0,1,2,1,0,0.0,1,0.506557,0.667322,0.979014
3067,их,3,3,15.0,12.0,1199.0,-1,3,-1,1,5,1,571,-1,-1,-1,5,0,0.0,0,0.425714,0.670254,0.979014
3127,них,3,3,16.0,2.0,1259.0,-1,1,-1,1,5,10,588,-1,-1,-1,0,0,0.0,0,0.413542,0.401902,0.979014
3362,ее,3,3,18.0,7.0,1494.0,-1,3,1,1,5,1,446,0,0,1,1,3,1490.0,1,0.494997,0.549569,0.979014
3452,она,3,3,19.0,0.0,1584.0,-1,0,1,1,5,22,670,-1,-1,1,6,3,1582.0,1,0.528053,0.667884,0.979014
3626,его,3,3,21.0,4.0,1758.0,-1,3,0,1,5,1,726,-1,-1,-1,0,3,1750.0,1,0.506557,0.583692,0.979014
3705,его,3,3,22.0,3.0,1837.0,-1,3,0,1,5,11,741,0,1,1,1,0,0.0,1,0.506557,0.505546,0.979014
3901,его,3,3,23.0,6.0,2033.0,-1,3,0,1,5,11,792,0,1,2,1,3,2031.0,1,0.506557,0.678267,0.979014
3923,его,3,3,23.0,7.0,2055.0,-1,3,0,1,5,11,792,0,1,2,1,3,2053.0,1,0.506557,0.678267,0.979014


In [396]:
topics = list(set(df.TOPIC_NUM.values))
topics
# all_anaphors = df.loc[(df['TOPIC_NUM'] == topic_num) & (df['POS'] == 5)]

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

In [587]:
# def anaphore_in_sent(sent_num, topic_num) это потом
# изначально отталкиваемся от номеров топиков, итерация по ним
# topics = list(set(df.TOPIC_NUM.values)) это выше
anf_len = 0
ant_len = 0
ann_clusters = []
my_clusters = []
post_num = 0
for topic_num in topics:
  # print(topics, 'topics')
  # if post_num == 3:
  #   break
  # извлекаем посты топика
  posts_in_topic = list(set(df.loc[df['TOPIC_NUM'] == topic_num].POST_NUM))
  # sents_in_topic = list(set(df.loc[df['TOPIC_NUM'] == topic_num].SENT_NUM))


  for post_num in posts_in_topic:
    sents_in_post = list(set(df.loc[(df['TOPIC_NUM'] == topic_num) \
                                    & (df['POST_NUM'] == post_num)].SENT_NUM))
    # if post_num == 3:
    #   break

    for sent_num in sents_in_post:

      sent = df.loc[(df['TOPIC_NUM'] == topic_num)  & (df['SENT_NUM'] == sent_num) \
                    & (df['POST_NUM'] == post_num)]
      # собираем анафоры и антецеденты
      current_anaphors = sent.loc[(df['POS'] == 5) &  (df['PERSON'] == 1)]
      # current_antecedents = sent.loc[(sent['NER'] > 1)]
      current_antecedents = sent.loc[(sent['NER'] == 1)]
      anf_len += len(current_anaphors)
      ant_len += len(current_antecedents)
      # если нет анафоров, продолжать не имеет смысла
      
      if not list(current_anaphors.TOKEN):
        continue

      # если в предложении есть один антецедент, самый простой случай
      if len(list(current_antecedents.TOKEN)) >= 1:
        for anfr in current_anaphors.values:
          # проверяем сходимость по числу
          if anfr[-4] == current_antecedents.NUMBER.values[0]:
            # добавляем в список размеченный класс
            ann_clusters.append(anfr[-5])
            # и класс который определило правило(антецедента из предложения)
            my_clusters.append(current_antecedents.TOKEN_NUM.values[0])
          else:
            # в противном случае присваиваем нулевой класс
            ann_clusters.append(anfr[-5])
            # и класс, который определило правило(антецедента из предложения)
            my_clusters.append(0.0)

      # если больше одного!!
          
      # если в предложении его нет =(
      elif len(list(current_antecedents.TOKEN)) == 0:
        # ищем в посте в предложениях выше
        post = df.loc[(df['TOPIC_NUM'] == topic_num) \
                      & (df['POST_NUM'] == post_num) \
                      & (df['SENT_NUM'] <= sent_num)]

        current_antecedents = post.loc[(post['NER'] == 1) ]

        # если его нету в посте, выбираем первый же антецедент из предыдущих постов
        # топика(сверху вниз)
        if not list(current_antecedents.TOKEN):
          topic = df.loc[(df['TOPIC_NUM'] == topic_num)]
          current_antecedents = topic.loc[(topic['NER'] == 1) ]
          # проверяем, а антецеденты-то есть?
          if not current_antecedents.TOKEN.values.any():
              # присваиваем нулевой класс, если нет даже в топике
              ann_clusters.append(anfr[-5])
              # и класс, который определило правило(антецедента из предложения)
              # для оценки
              my_clusters.append(0.0)
              continue
          # ну и если они есть
          # это можно убрать в отдельную функцию
          for anfr in current_anaphors.values:
            # проверяем сходимость по числу
            # try:
            if anfr[-4] == current_antecedents.NUMBER.values[0]:
              # добавляем в список размеченный класс
              ann_clusters.append(anfr[-5])
              # и класс который определило правило(антецедента из предложения)
              my_clusters.append(current_antecedents.TOKEN_NUM.values[0])
              # print(ann_clusters, my_clusters, 'am I here?')
            else:
              # в противном случае присваиваем нулевой класс
              ann_clusters.append(anfr[-5])
              # и класс, который определило правило(антецедента из предложения)
              my_clusters.append(0.0)


        # если антецедент есть в посте раньше, выбираем первый с конца [::-1]
        else:
          # это можно убрать в отдельную функцию
          for anfr in current_anaphors.values:
            # проверяем сходимость по числу, если число вообще есть
          
            if anfr[-4] == current_antecedents.NUMBER.values[-1]:
              # добавляем в список размеченный класс
              ann_clusters.append(anfr[-5])
              # и класс который определило правило(антецедента из предложения)
              my_clusters.append(current_antecedents.TOKEN_NUM.values[-1])

            else:
              # в противном случае присваиваем нулевой класс
              ann_clusters.append(anfr[-5])
              # и класс, который определило правило(антецедента из предложения)
              my_clusters.append(0.0)






      # print(list(current_anaphors.TOKEN), list(current_anaphors.TOKEN), list(current_anaphors.COREFERENCE_CLUSTER) )
      # print(list(current_antecedents.TOKEN), list(current_antecedents.TOKEN), list(current_antecedents.COREFERENCE_CLUSTER) )
print(my_clusters)
print('---')
print(ann_clusters)
print('---')
print(len(my_clusters), len(ann_clusters))
print(anf_len, ant_len)




[2.0, 540.0, 11.0, 11.0, 108.0, 182.0, 182.0, 182.0, 182.0, 295.0, 349.0, 558.0, 658.0, 658.0, 755.0, 829.0, 829.0, 829.0, 829.0, 942.0, 996.0, 1205.0, 0.0, 0.0, 0.0, 0.0, 2.0, 679.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 1483.0, 0.0, 1750.0, 1844.0, 2031.0, 2053.0, 2156.0, 0.0, 0.0, 0.0, 0.0, 3125.0, 3270.0, 3488.0, 3511.0, 0.0, 3619.0, 5.0, 0.0, 114.0, 0.0, 0.0, 174.0, 0.0, 0.0, 0.0, 365.0, 0.0, 527.0, 0.0, 10.0, 27.0, 41.0, 41.0, 2.0, 135.0, 135.0, 218.0, 239.0, 256.0, 270.0, 270.0, 2.0, 2.0, 2.0, 447.0, 2.0, 521.0, 521.0, 521.0, 0.0, 0.0, 655.0, 0.0, 0.0, 2.0, 2.0, 0.0, 1369.0, 1455.0, 1516.0, 1609.0, 1657.0, 1683.0, 1722.0, 1770.0, 0.0, 1905.0, 0.0, 1984.0, 0.0, 1993.0, 1993.0, 1993.0, 0.0, 2071.0, 0.0, 0.0, 0.0, 40.0, 164.0, 0.0, 164.0, 0.0, 0.0, 410.0, 534.0, 0.0, 534.0, 0.0, 707.0, 707.0, 801.0, 0.0, 871.0, 881.0, 990.0, 1047.0, 1350.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 322.0, 174.0, 174.0, 27.0, 27.0, 122.0, 165.0, 263.0, 0.0, 513.0, 556.0, 654.0, 0.0, 8

In [588]:
rule_eval(my_clusters, ann_clusters)

52  true positive
59  false positive
55  true negative
28  false negative
---------
0.46846846846846846  presicion
0.65  recall
0.5445026178010471  f-score


# **МОДЕЛИ.**

Факторизуем троковые данные, чтобы вместо строковых представлений классов были численные.

In [399]:
for_factorize = 'ANIMACY 	CASE	GENDER	PERSON	POS	DEPENDENCY	HEAD	HEAD_ANIMACY	HEAD_CASE	HEAD_GENDER	HEAD_POS NUMBER'.split()
for_factorize

['ANIMACY',
 'CASE',
 'GENDER',
 'PERSON',
 'POS',
 'DEPENDENCY',
 'HEAD',
 'HEAD_ANIMACY',
 'HEAD_CASE',
 'HEAD_GENDER',
 'HEAD_POS',
 'NUMBER']

In [400]:
_, lbl = pd.factorize(df.ANIMACY)
print(lbl, )
_

Index(['inan', 'anim'], dtype='object')


array([-1,  0,  1, ..., -1, -1,  1])

Делаем словарь с маппингом, чтобы можно было достать нужное значение по индексу. В значении -1 хранится None

In [401]:
mapping_dict = {}
for col_name in for_factorize:
  col, lbl = pd.factorize(df[col_name])
  df[col_name] = col
  mapping_dict[col_name] = lbl
mapping_dict  

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


{'ANIMACY': Index(['inan', 'anim'], dtype='object'),
 'CASE': Index(['nomn', 'gent', 'datv', 'accs', 'loct', 'ablt', 'loc2', 'voct', 'gen2'], dtype='object'),
 'DEPENDENCY': Index(['ROOT', 'obj', 'appos', 'parataxis', 'conj', 'advmod', 'ccomp', 'iobj',
        'csubj', 'case', 'obl', 'det', 'nsubj', 'cc', 'discourse', 'fixed',
        'mark', 'amod', 'advcl', 'xcomp', 'nummod', 'nmod', 'nsubj:pass',
        'nummod:gov', 'cop', 'acl:relcl', 'acl', 'aux', 'flat:foreign', 'punct',
        'obl:agent', 'flat', 'aux:pass', 'orphan', 'expl', 'csubj:pass'],
       dtype='object'),
 'GENDER': Index(['masc', 'femn', 'neut'], dtype='object'),
 'HEAD': Index(['обсуждаем', 'препарат', 'рисперидон', 'рисполепт', 'знаю', 'стоило',
        'меня', 'нет', 'проблем', 'лекарством',
        ...
        'удобнее', 'иллюстрации', 'поиске', 'темам', 'посмотреть', 'случай',
        'описала', 'побочку', 'гемцитабина', 'прошлась'],
       dtype='object', length=2898),
 'HEAD_ANIMACY': Index(['inan', 'anim', 

In [402]:

# labels_dict = {}
# test_labels_dict = {}

# for i in X.columns:
#   X[i], lbl = pd.factorize(X[i])
#   labels_dict[i] = lbl

# for i in X_test.columns:
#   X_test[i], lbl = pd.factorize(X_test[i])
#   test_labels_dict[i] = lbl 

# y = cor_df['COREFERENCE_CLUSTER']
# y, y_label = pd.factorize(y)

# y_test = cor_df_test['COREFERENCE_CLUSTER']
# y_test, y_test_label = pd.factorize(y_test)

In [403]:
df.loc[ (df['NER'] == 3.0 )]

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
475,его,1,1,7.0,1.0,475.0,-1,3,0,1,5,11,199,0,2,2,1,3,2.0,1,0.506557,0.335296,0.981575
566,он,1,1,9.0,3.0,566.0,-1,0,0,1,5,12,231,-1,-1,0,0,3,536.0,1,0.493189,0.403330,0.968715
587,он,2,2,1.0,0.0,13.0,-1,0,0,1,5,12,235,-1,-1,0,0,3,11.0,1,0.493189,0.559345,0.968715
594,он,2,2,1.0,1.0,20.0,-1,0,0,1,5,12,242,-1,-1,-1,0,3,11.0,1,0.493189,0.578639,0.968715
736,него,2,2,3.0,5.0,162.0,-1,1,0,1,5,10,293,-1,-1,-1,0,3,108.0,1,0.506391,0.569367,0.973589
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14440,его,15,15,14.0,2.0,1848.0,-1,3,0,1,5,1,1175,-1,-1,-1,0,3,1827.0,1,0.506557,0.535808,0.973753
14478,него,15,15,14.0,6.0,1886.0,-1,1,0,1,5,21,2453,-1,-1,-1,15,3,1867.0,1,0.506391,0.498309,0.973753
14698,химии,15,15,17.0,3.0,2106.0,0,1,1,-1,1,0,1697,0,1,1,1,3,0.0,1,0.603188,0.482761,0.974329
14733,него,15,15,17.0,8.0,2141.0,-1,1,0,1,5,10,2278,-1,-1,1,0,3,2138.0,1,0.506391,0.569898,0.966645


In [404]:
df.loc[(df['NER'] == 1.0) | (df['NER'] == 2.0 )]

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
2,рисперидон,1,1,0.0,0.0,2.0,1,0,0,-1,1,2,1,0,0,0,1,1,2.0,1,0.769174,0.865285,0.952796
3,рисполепт,1,1,0.0,0.0,3.0,0,1,1,-1,1,3,2,1,0,0,1,1,3.0,0,0.775658,0.556566,0.952796
4,сперидан,1,1,0.0,0.0,4.0,-1,-1,0,-1,2,4,3,0,1,1,1,1,4.0,1,0.750261,0.634239,0.952796
21,лекарством,1,1,1.0,1.0,21.0,0,5,2,-1,1,4,7,-1,-1,-1,3,2,0.0,1,0.484399,0.548811,0.952796
142,рисполепта,1,1,1.0,13.0,142.0,0,0,1,-1,1,21,69,0,2,1,1,1,142.0,1,0.734191,0.515975,0.966061
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14759,hifu,15,15,17.0,12.0,2167.0,-1,-1,-1,-1,-1,1,1903,-1,-1,-1,0,1,2167.0,-1,0.512034,0.514084,0.966645
14807,абраксан,15,15,18.0,3.0,2215.0,1,1,1,-1,1,1,2480,-1,-1,-1,0,1,2215.0,0,0.703071,0.408048,0.957845
14809,гемцитабином,15,15,18.0,3.0,2217.0,0,5,0,-1,1,21,2470,1,1,1,1,1,2217.0,1,0.615143,0.558340,0.957845
14856,фолфиринокс,15,15,18.0,8.0,2264.0,0,0,0,-1,1,21,1910,0,0,1,1,1,2264.0,1,0.724853,0.429119,0.953817


Разделим датасет на тестовый и тренировочный. for_train, for_test 

In [405]:
# train, test = train_test_split(df.loc[(df['NER'] == 1.0) | (df['NER'] == 2.0 ) | (df['NER'] == 3.0 )], test_size=0.2)
# train, test = train_test_split(df, test_size=0.2)
train, test = train_test_split(df.loc[(df['NER'] == 2.0 ) | (df['NER'] == 3.0 )], test_size=0.2)

In [406]:
train = train.reset_index()

In [407]:
test = test.reset_index()

In [408]:
test.head(5)

,index,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
0,7682,он,5,5,41.0,3.0,1484.0,-1,0,0,1,5,12,1427,-1,-1,0,0,3,1455.0,1,0.493189,0.566620,0.973646
1,11469,обезболивающие,9,9,0.0,34.0,300.0,-1,0,-1,-1,14,26,1557,0,2,1,1,2,301.0,0,0.578929,0.529434,0.977045
2,5889,препарат,4,4,6.0,6.0,336.0,0,0,0,-1,1,1,1158,-1,-1,-1,0,2,287.0,1,0.560645,0.504357,0.988290
3,9691,препараты,7,7,5.0,0.0,1152.0,0,0,0,-1,1,12,619,-1,-1,-1,13,2,0.0,0,0.490358,0.387470,0.784757
4,6254,препарата,5,5,3.0,1.0,56.0,0,1,0,-1,1,21,1264,0,1,2,1,2,41.0,1,0.487741,0.589720,0.988290


In [409]:
# test = test.iloc[:,:24].drop(['TOKEN', 'index'], axis=1)
# test = test.drop(['TOKEN', 'index'], axis=1)

In [410]:
# test.loc[test['NER'] != 0]

In [411]:
# ['TOPIC_NUM',	'POST_NUM',	'TOKEN_NUM',	'GENDER',	'PERSON', 'TOKEN', 'TOKEN_VECT', 'IS_ANSWER', 'SENT_NUM', 'HEAD_VECT', 'HEAD_CHILDS_VECT', 
#  'index', 'HEAD_ANIMACY',	'HEAD_CASE',	'HEAD_GENDER', 'ANIMACY',	'CASE',	'POS',	'DEPENDENCY',	'HEAD',	'HEAD_POS']

In [412]:
# best set

# X, y, X_test, y_test = train.drop(['TOKEN',  'HEAD_VECT', 'POST_NUM', 'IS_ANSWER', 'PERSON', 'HEAD_CHILDS_VECT', 'TOPIC_NUM', 'CASE', 'SENT_NUM', 'DEPENDENCY',  'index', 'HEAD_ANIMACY',	'HEAD_CASE',	'HEAD_GENDER', 'ANIMACY',				'HEAD',	'HEAD_POS'], axis=1).iloc[:,:18], \
# train.COREFERENCE_CLUSTER, test.drop(['TOKEN',  'HEAD_VECT', 'POST_NUM', 'IS_ANSWER', 'PERSON', 'HEAD_CHILDS_VECT', 'TOPIC_NUM', 'CASE', 'SENT_NUM', 'DEPENDENCY',    'index', 'HEAD_ANIMACY',	'HEAD_CASE',	'HEAD_GENDER', 'ANIMACY',			'HEAD',	'HEAD_POS'], axis=1).iloc[:,:18], test.COREFERENCE_CLUSTER

Делаем наборы для обучения моделей.

In [413]:
X, y, X_test, y_test = train.drop(['TOKEN','GENDER','NUMBER', 'POS', 'HEAD_VECT', 'POST_NUM', 'IS_ANSWER', 'PERSON', 'TOPIC_NUM', 'CASE', 'SENT_NUM', 'DEPENDENCY',  'index', 'HEAD_ANIMACY',	'HEAD_CASE',	'HEAD_GENDER', 'ANIMACY',	'HEAD',	'HEAD_POS'], axis=1).iloc[:,:18], \
train.COREFERENCE_CLUSTER, test.drop(['TOKEN','GENDER', 'NUMBER', 'POS', 'HEAD_VECT', 'POST_NUM', 'IS_ANSWER', 'PERSON', 'TOPIC_NUM', 'CASE', 'SENT_NUM', 'DEPENDENCY',    'index', 'HEAD_ANIMACY',	'HEAD_CASE',	'HEAD_GENDER', 'ANIMACY',	'HEAD',	'HEAD_POS'], axis=1).iloc[:,:18], test.COREFERENCE_CLUSTER

In [414]:
# X, y, X_test, y_test = train.drop(['TOKEN', 'index'], axis=1).iloc[:,:24], train.COREFERENCE_CLUSTER, test.drop(['TOKEN', 'index'], axis=1).iloc[:,:24], test.COREFERENCE_CLUSTER

In [415]:
X.NER = [float(i) for i in X.NER]
X_test.NER = [float(i) for i in X_test.NER]


In [416]:
y

0      2058.0
1       449.0
2         0.0
3       658.0
4         0.0
        ...  
257       0.0
258     564.0
259      58.0
260     135.0
261     806.0
Name: COREFERENCE_CLUSTER, Length: 262, dtype: float64

Обучаем модели.

In [417]:
clf = tree.DecisionTreeClassifier(max_depth=2000, )
clf.fit(X, y)
clf.score(X_test, y_test)

0.7424242424242424

In [418]:
clf_entropy = tree.DecisionTreeClassifier(criterion='entropy', max_depth=2000)
clf_entropy.fit(X, y)
clf_entropy.score(X_test, y_test)

0.7121212121212122

In [419]:
randf_clf = RandomForestClassifier(max_depth=3000, n_estimators=600, bootstrap=True, )
randf_clf.fit(X, y)
randf_clf.score(X_test, y_test)

0.696969696969697

In [420]:
regr = linear_model.LinearRegression()
regr.fit(X, y)
regr.score(X_test, y_test)


1.0

In [421]:
svm_clf = svm.SVC(decision_function_shape='ovo', cache_size=500)
svm_clf.fit(X, y)
svm_clf.score(X_test, y_test)

0.3787878787878788

In [422]:

knn_clf = KNeighborsClassifier(n_neighbors=10, weights='distance',algorithm='ball_tree',leaf_size=5, metric='minkowski', p=1).fit(X, y)
knn_clf.score(X_test, y_test)

0.696969696969697

In [423]:
# from sklearn.naive_bayes import MultinomialNB
# naive_clf = MultinomialNB().fit(X, y)


# **ОЦЕНКА МОДЕЛЕЙ**

In [424]:
def my_eval(X_predicted, y):
  
  tp = []
  fp = []
  tn = []
  fn = []

  for i in range(len(X_predicted)):
    if X_predicted[i] != 0.0:
      if X_predicted[i] == y_test[i]:
        tp.append(i)
      else:
        fp.append(i)
    # берем только те синглтоны, которые относятся к номинативным или прономинальным группам.
    elif X_predicted[i] == 0.0 and test.iloc[:,22:23].iloc[[i]].values[0][0] in (2.0, 3.0):
      if X_predicted[i] == y_test[i]:
        tn.append(i)
      else:
        fn.append(i)
  try:
    P = len(tp)/(len(tp) + len(fp))
  except ZeroDivisionError:
    P = 0
  try:
    R = len(tp)/(len(tp) + len(fn))
  except ZeroDivisionError:
    R = 0
  try:
    F = 2*P*R/(P+R)
  except ZeroDivisionError:
    F = 0

  print(len(tp), ' true positive')
  print(len(fp), ' false positive')
  print(len(tn), ' true negative')
  print(len(fn), ' false negative')
  print('---------')
  print(P,' presicion')
  print(R, ' recall')
  print(F, ' f-score')



In [425]:
pred = clf.predict(X_test)
pred_randf = randf_clf.predict(X_test)
pred_entropy = clf_entropy.predict(X_test)
pred_regr = regr.predict(X_test)
pred_svm = svm_clf.predict(X_test)
pred_knn = knn_clf.predict(X_test)
# naive_pred = naive_clf.predict(X_test)

In [426]:
# my_eval(list(naive_pred), list(y_test))

In [427]:
my_eval(list(pred_knn), list(y_test))

25  true positive
18  false positive
0  true negative
0  false negative
---------
0.5813953488372093  presicion
1.0  recall
0.7352941176470588  f-score


In [428]:
my_eval(list(pred_entropy), list(y_test))

25  true positive
19  false positive
0  true negative
0  false negative
---------
0.5681818181818182  presicion
1.0  recall
0.7246376811594203  f-score


In [429]:
my_eval(list(pred), list(y_test))

27  true positive
17  false positive
0  true negative
0  false negative
---------
0.6136363636363636  presicion
1.0  recall
0.7605633802816901  f-score


In [430]:
my_eval(list(pred_randf), list(y_test))

24  true positive
20  false positive
0  true negative
0  false negative
---------
0.5454545454545454  presicion
1.0  recall
0.7058823529411764  f-score


In [431]:
my_eval(list(pred_regr), list(y_test))

6  true positive
60  false positive
0  true negative
0  false negative
---------
0.09090909090909091  presicion
1.0  recall
0.16666666666666669  f-score


In [432]:
my_eval(list(pred_svm), list(y_test))

3  true positive
18  false positive
0  true negative
0  false negative
---------
0.14285714285714285  presicion
1.0  recall
0.25  f-score


In [433]:
# df['TOPIC_NUM'] == 1
# df.loc[df['TOPIC_NUM'] == 1]

In [434]:
df.drop(['TOKEN','GENDER','NUMBER', 'POS', 'HEAD_VECT', 'POST_NUM', 'IS_ANSWER', 'PERSON', \
         'TOPIC_NUM', 'CASE', 'SENT_NUM', 'DEPENDENCY',  'HEAD_ANIMACY',	'HEAD_CASE',	\
         'HEAD_GENDER', 'ANIMACY',	'HEAD',	'HEAD_POS'], axis=1).iloc[:,:18]

,TOKEN_NUM,NER,COREFERENCE_CLUSTER,TOKEN_VECT,SENT_VECT
0,0.0,0,0.0,0.495954,0.952796
1,1.0,0,0.0,0.560645,0.952796
2,2.0,1,2.0,0.769174,0.952796
3,3.0,1,3.0,0.775658,0.952796
4,4.0,1,4.0,0.750261,0.952796
...,...,...,...,...,...
17195,606.0,0,0.0,0.490155,0.993070
17196,607.0,0,0.0,0.536465,0.993070
17197,608.0,0,0.0,0.671096,0.993070
17198,609.0,0,0.0,0.612565,0.993070


In [435]:
len(df.loc[(df['POS'] == 5) & (df['NER'] > 1)].TOKEN)

114

In [436]:
df.loc[(df['COREFERENCE_CLUSTER'] != 0) & (df['NER'] > 1)]

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
366,препарата,1,1,2.0,5.0,366.0,0,1,0,-1,1,21,154,0,1,1,1,2,344.0,1,0.487741,0.431597,0.947810
421,препарат,1,1,5.0,0.0,421.0,0,0,0,-1,1,0,1,0,0,0,1,2,2.0,1,0.560645,0.865285,0.925857
429,препарат,1,1,6.0,0.0,429.0,0,0,0,-1,1,12,179,-1,-1,-1,0,2,2.0,1,0.560645,0.521736,0.925857
446,последнему,1,1,6.0,2.0,446.0,-1,2,0,-1,10,7,184,-1,-1,-1,0,2,442.0,1,0.595085,0.511423,0.761660
454,препарат,1,1,7.0,0.0,454.0,0,0,0,-1,1,12,189,-1,-1,-1,0,2,2.0,1,0.560645,0.499556,0.786368
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14427,него,15,15,14.0,2.0,1835.0,-1,1,0,1,5,10,195,-1,-1,-1,9,3,1827.0,1,0.506391,0.580910,0.973753
14440,его,15,15,14.0,2.0,1848.0,-1,3,0,1,5,1,1175,-1,-1,-1,0,3,1827.0,1,0.506557,0.535808,0.973753
14478,него,15,15,14.0,6.0,1886.0,-1,1,0,1,5,21,2453,-1,-1,-1,15,3,1867.0,1,0.506391,0.498309,0.973753
14733,него,15,15,17.0,8.0,2141.0,-1,1,0,1,5,10,2278,-1,-1,1,0,3,2138.0,1,0.506391,0.569898,0.966645


# **ПОПАРНЫЙ МЕТОД**

In [437]:
def pair_eval(X_predicted, y):
  
  tp = []
  fp = []
  tn = []
  fn = []

  for i in range(len(X_predicted)):
    if X_predicted[i] != 0.0:
      if X_predicted[i] == y[i]:
        tp.append(i)
      else:
        fp.append(i)
    # берем только те синглтоны, которые относятся к номинативным или прономинальным группам.
    elif X_predicted[i] == 0.0:
      if X_predicted[i] == y[i]:
        tn.append(i)
      else:
        fn.append(i)
  try:
    P = len(tp)/(len(tp) + len(fp))
  except ZeroDivisionError:
    P = 0
  try:
    R = len(tp)/(len(tp) + len(fn))
  except ZeroDivisionError:
    R = 0
  try:
    F = 2*P*R/(P+R)
  except ZeroDivisionError:
    F = 0

  print(len(tp), ' true positive')
  print(len(fp), ' false positive')
  print(len(tn), ' true negative')
  print(len(fn), ' false negative')
  print('---------')
  print(P,' presicion')
  print(R, ' recall')
  print(F, ' f-score')

In [559]:
topics = list(set(df.TOPIC_NUM.values))
topics


pair_anaphors = []
pair_antecedents = []
relation_list = []
counter = 0
ants_with_anaph = 0
len_anapfors = 0
for topic_num in topics:

  # all_anaphors = df.loc[(df['TOPIC_NUM'] == topic_num) & (df['NER'] > 1)]

  # all_antecedents = df.loc[(df['TOPIC_NUM'] == topic_num) & (df['NER'] == 1)]

  all_anaphors = df.loc[(df['TOPIC_NUM'] == topic_num) & (df['POS'] == 5)]

  all_antecedents = df.loc[(df['TOPIC_NUM'] == topic_num) & (df['NER'] == 1)]
  
  len_anapfors += len(all_anaphors)
  for anaphor in all_anaphors.values:
    # чтобы примеры влезли в память, и модель не переобучилась на антипримерах,
    # возьмем один положительный и два отрицательных класса для каждого анафора
    counter = 0
    is_relation = 0
    for antecedent in all_antecedents.values:
      

      # print(anaphor[-5], antecedent[5])
      if anaphor[-5] == antecedent[5]:
        ants_with_anaph += 1
        is_relation = 1
        # такие срезы, чтобы убрать лишние данные, и в том числе разметку с ответами
        pair_anaphors.append(list(anaphor)[3:6] )# + list(anaphor)[17:18]\
                             # + list(anaphor)[19:23])
        
        pair_antecedents.append(list(antecedent)[3:6]) # + list(antecedent)[17:18]\
                             # + list(antecedent)[19:23])
        
        relation_list.append(is_relation)
      else:
        if counter != 1:

          pair_anaphors.append(list(anaphor)[3:6] )# + list(anaphor)[17:18]\
                              # + list(anaphor)[19:23])
          
          pair_antecedents.append(list(antecedent)[3:6]) # + list(antecedent)[17:18]\
                              # + list(antecedent)[19:23])
          
          relation_list.append(is_relation)
          counter = 1

  # counter += 1
print(len_anapfors)
print('---')
print(pair_anaphors[0][0], pair_antecedents[0][0], relation_list[0])
len(pair_anaphors), len(pair_antecedents), len(relation_list)

725
---
1.0 0.0 0


(698, 698, 698)

In [560]:
# можно посмотреть, что получилось
relation_list[:20]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [561]:
pair_anaphors[0]

[1.0, 0.0, 9.0]

In [562]:
# проверим, третьим элементом должен быть номер токена
len([i for i in relation_list if i==1])

88

In [563]:
# все сходится. мы взяли колонки с номерами поста, предложения и токена,\
# колонку NER, число, и вектора
df.loc[df.TOKEN_NUM == 21.0].iloc[[0]]

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
21,лекарством,1,1,1.0,1.0,21.0,0,5,2,-1,1,4,7,-1,-1,-1,3,2,0.0,1,0.484399,0.548811,0.952796


In [564]:
cosine = [cosine_similarity([pair_anaphors[i]], [pair_antecedents[i]])[0][0] for i in range(len(pair_anaphors))]
# pair_df = pd.DataFrame(columns=['anaphs', 'ants', 'rel'])
pair_df = pd.DataFrame(columns=['cosine', 'rel'])

# pair_df['anaphs'] = pair_anaphors
# pair_df['ants'] = pair_antecedents
pair_df['cosine'] = cosine
pair_df['rel'] = relation_list

print(pair_df.loc[pair_df['cosine'] > 1])
print(pair_df.loc[pair_df['rel'] > 1])
pair_df

Empty DataFrame
Columns: [cosine, rel]
Index: []
Empty DataFrame
Columns: [cosine, rel]
Index: []


,cosine,rel
0,0.993884,0
1,0.993127,0
2,0.995440,0
3,0.996729,0
4,0.997029,0
...,...,...
693,0.996801,0
694,0.996870,0
695,0.996619,0
696,0.999999,1


In [565]:
pair_df.loc[pair_df.rel != 0]

,cosine,rel
26,0.999889,1
27,0.999889,1
33,0.999994,1
36,0.999904,1
38,0.997928,1
...,...,...
643,0.999996,1
651,0.999984,1
669,0.999999,1
684,1.000000,1


In [566]:
train, test = train_test_split(pair_df, test_size=0.3, shuffle=True)
train = train.reset_index()
test = test.reset_index()

In [567]:
X, y, X_test, y_test = train.iloc[:, 1:2], train.rel, test.iloc[:, 1:2], test.rel


In [568]:
train.iloc[:1, 1:2]

,cosine
0,0.999997


In [569]:
test.iloc[:1, 1:2]

,cosine
0,0.995267


In [570]:
regr = linear_model.LinearRegression()
regr.fit(X, y)


print('-----')
pred = regr.predict(X_test)
pair_eval(pred, y_test)

-----
0  true positive
210  false positive
0  true negative
0  false negative
---------
0.0  presicion
0  recall
0  f-score


In [571]:
knn_clf = KNeighborsClassifier(n_neighbors=10, weights='distance',algorithm='ball_tree',leaf_size=5, metric='minkowski', p=1)
knn_clf.fit(X, y)


print('-----')
pred = knn_clf.predict(X_test)
pair_eval(pred, y_test)

-----
16  true positive
7  false positive
177  true negative
10  false negative
---------
0.6956521739130435  presicion
0.6153846153846154  recall
0.6530612244897959  f-score


In [572]:
dtclf = tree.DecisionTreeClassifier(max_depth=2000, )
dtclf.fit(X, y)


print('-----')
pred = dtclf.predict(X_test)
pair_eval(pred, y_test)


-----
15  true positive
8  false positive
176  true negative
11  false negative
---------
0.6521739130434783  presicion
0.5769230769230769  recall
0.6122448979591837  f-score


In [573]:
clf = tree.DecisionTreeClassifier(criterion='entropy', max_depth=2000)
clf.fit(X, y)


print('-----')
pred = clf.predict(X_test)
pair_eval(pred, y_test)

-----
15  true positive
8  false positive
176  true negative
11  false negative
---------
0.6521739130434783  presicion
0.5769230769230769  recall
0.6122448979591837  f-score


In [574]:
clf = RandomForestClassifier(max_depth=3000, n_estimators=600, bootstrap=True, )
clf.fit(X, y)


print('-----')
pred = clf.predict(X_test)
rule_eval(pred, y_test)

-----
17  true positive
8  false positive
176  true negative
9  false negative
---------
0.68  presicion
0.6538461538461539  recall
0.6666666666666666  f-score


In [541]:
# knn_clf = KNeighborsClassifier(n_neighbors=10, weights='distance',algorithm='ball_tree',leaf_size=5, metric='minkowski', p=1).fit(X, y)
# knn_clf.score(X_test, y_test)

In [542]:
# clf = tree.DecisionTreeClassifier(max_depth=2000, )
# clf.fit(X, y)
# clf.score(X_test, y_test)